In [1]:
import ast
import re
from event_loop.preprocessing.dataframe import *

import metrics
import numpy as np
import pandas as pd
import pm4py 

In [2]:
def numeric_to_string(x):
    return str(x)

# HR data in data/Train/R1 is missing frame.number. We take another (already filtered) dataset and apply our feature extraction to this one
df_train_in = pd.read_csv('../../data_v3/hr_extended_features.csv', converters={"MessageAttributes": ast.literal_eval, "InstanceNumber": numeric_to_string},parse_dates=["synthetic_sniff_time"])

In [13]:
df_train_r1 = pd.read_csv('../../data/TRAIN/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

In [3]:
df_gt = pd.read_csv("../../data_v3/hr_ground_truth.csv",dtype={"bp_id": str})
df_times = pd.read_csv("../../data_v3/hr_il_tagged.csv", usecols=["frame.number","synthetic_sniff_time"],
                       parse_dates=["synthetic_sniff_time"])
df_gt = df_gt.merge(df_times, how="left", left_on="start", right_on="frame.number")

In [4]:
df_train = pre_process(df_train_in)

In [89]:
df_train

,BusinessActivity,InstanceNumber,frame.number,sniff_time,synthetic_sniff_time,event_with_roles,request_method_call,file_data,pgsql.query,pgsql.target,selective_file_data,origin_method,origin_file_data,activity_cat,case:concept:name,concept:name,time:timestamp
0,GenerateJobApplicationActivity,1,26,2020-05-12 09:57:23.762710+00:00,2020-05-13 10:07:18.092710+00:00,End Point (HR Manager)->Odoo Application:[Http...,version,[],,,version,,,End Point (HR Manager)->Odoo Application:[Http...,1,End Point (HR Manager)->Odoo Application:[Http...,2020-05-13 10:07:18.092710+00:00
1,GenerateJobApplicationActivity,1,127,2020-05-12 09:57:28.092427+00:00,2020-05-13 22:08:59.592427+00:00,Odoo Application->End Point (HR Manager):[Http...,server_version,"[server_version, 12.0-20190820, server_version...",,,server_version,version,version,Odoo Application->End Point (HR Manager):[Http...,1,Odoo Application->End Point (HR Manager):[Http...,2020-05-13 22:08:59.592427+00:00
2,GenerateJobApplicationActivity,1,135,2020-05-12 09:57:28.094494+00:00,2020-05-13 22:09:20.264494+00:00,End Point (HR Manager)->Odoo Application:[Http...,authenticate,"[odoo01, dana.wireless@gmail.com, 123456789]",,,,,,End Point (HR Manager)->Odoo Application:[Http...,1,End Point (HR Manager)->Odoo Application:[Http...,2020-05-13 22:09:20.264494+00:00
3,GenerateJobApplicationActivity,1,192,2020-05-12 09:57:28.580005+00:00,2020-05-13 23:30:15.860005+00:00,Odoo Application->db Server/Mail Server:[Pgsql...,,,"INSERT INTO ""res_users_log"" (""id"", ""create_uid...",res_users_log,,,,Odoo Application->db Server/Mail Server:[Pgsql...,1,Odoo Application->db Server/Mail Server:[Pgsql...,2020-05-13 23:30:15.860005+00:00
4,GenerateJobApplicationActivity,1,216,2020-05-12 09:57:28.729685+00:00,2020-05-13 23:55:12.809685+00:00,Odoo Application->End Point (HR Manager):[Http...,IsNumber,[6],,,IsNumber,authenticate,,Odoo Application->End Point (HR Manager):[Http...,1,Odoo Application->End Point (HR Manager):[Http...,2020-05-13 23:55:12.809685+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33068,ResumeReviewActivity,250,729,2020-05-14 00:17:45.346366+00:00,2020-05-14 04:15:16.416366+00:00,Odoo Application->db Server/Mail Server:[Pgsql...,,,"UPDATE ""mail_message_res_partner_needaction_re...",mail_message_res_partner_needaction_rel,,,,Odoo Application->db Server/Mail Server:[Pgsql...,250,Odoo Application->db Server/Mail Server:[Pgsql...,2020-05-14 04:15:16.416366+00:00
33069,ResumeReviewActivity,250,766,2020-05-14 00:17:45.361497+00:00,2020-05-14 04:17:47.741497+00:00,Odoo Application->End Point (HR Manager):[Http...,IsNumber,[1],,,IsNumber,execute_kw,hr.applicant_write_stage_id_2,Odoo Application->End Point (HR Manager):[Http...,250,Odoo Application->End Point (HR Manager):[Http...,2020-05-14 04:17:47.741497+00:00
33070,ResumeReviewActivity,250,774,2020-05-14 00:17:45.363203+00:00,2020-05-14 04:18:04.803203+00:00,End Point (HR Manager)->Odoo Application:[Http...,execute_kw,"[odoo01, 6, 123456789, hr.applicant, archive_a...",,,hr.applicant_archive_applicant,,,End Point (HR Manager)->Odoo Application:[Http...,250,End Point (HR Manager)->Odoo Application:[Http...,2020-05-14 04:18:04.803203+00:00
33071,ResumeReviewActivity,250,840,2020-05-14 00:17:45.427437+00:00,2020-05-14 04:28:47.207437+00:00,Odoo Application->db Server/Mail Server:[Pgsql...,,,"UPDATE ""hr_applicant"" SET ""active""=false,""writ...",hr_applicant,,,,Odoo Application->db Server/Mail Server:[Pgsql...,250,Odoo Application->db Server/Mail Server:[Pgsql...,2020-05-14 04:28:47.207437+00:00


In [8]:
df_train["activity_cat"] = df_train["event_with_roles"] + df_train["file_data"].astype(str) + df_train["pgsql.query"]

In [91]:
df_train.to_csv("low_level_data_HR.csv")

In [62]:
petri_net_ref, im_ref, fm_ref = pm4py.discover_petri_net_inductive(df_train[df_train["InstanceNumber"] == "1"], case_id_key="InstanceNumber", activity_key="activity_cat",timestamp_key="synthetic_sniff_time")

In [98]:
df = pm4py.format_dataframe(df_train[["InstanceNumber", "activity_cat", "synthetic_sniff_time"]], case_id="InstanceNumber", activity_key="activity_cat",timestamp_key="synthetic_sniff_time")
log = pm4py.convert_to_event_log(df)
pm4py.write_xes(log, "exported.xes")

/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/pm4py/utils.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[constants.CASE_CONCEPT_NAME] = df[case_id]
/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/pm4py/utils.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[xes_constants.DEFAULT_NAME_KEY] = df[activity_key]
/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/pm4py/utils.py:85: SettingWithCop

exporting log, completed traces ::   0%|          | 0/250 [00:00<?, ?it/s]

In [97]:
df = pm4py.format_dataframe(df_gt[["bp_id", "activity_name", "synthetic_sniff_time"]], case_id="bp_id", activity_key="activity_name",timestamp_key="synthetic_sniff_time")
log = pm4py.convert_to_event_log(df)
pm4py.write_xes(log, "exported_gt.xes")

/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/pm4py/utils.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[constants.CASE_CONCEPT_NAME] = df[case_id]
/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/pm4py/utils.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[xes_constants.DEFAULT_NAME_KEY] = df[activity_key]
/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/pm4py/utils.py:85: SettingWithCop

exporting log, completed traces ::   0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
df_train_in

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FileName,BusinessActivity,frame.number,sniff_time_str,sniff_time,InstanceNumber,synthetic_sniff_time,...,IsEndActivity,ActivityAction,IsEndBP,IsStartBP,selective_filter_data,origin_request_method,origin_selective_filter_data,is_archive_applicant,pgsql.query,pgsql.target
0,0,0,0,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,26,2020-05-12 09:57:23.762710,2020-05-12 09:57:23.762710,1,2020-05-13 10:07:18.092710,...,0.0,Activity Start,False,True,version,NaN,NaN,False,None,None
1,1,1,1,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,127,2020-05-12 09:57:28.092427,2020-05-12 09:57:28.092427,1,2020-05-13 22:08:59.592427,...,0.0,NoAction,False,False,NaN,version,version,False,None,None
2,2,2,2,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,135,2020-05-12 09:57:28.094494,2020-05-12 09:57:28.094494,1,2020-05-13 22:09:20.264494,...,0.0,NoAction,False,False,NaN,NaN,NaN,False,None,None
3,3,3,3,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,192,2020-05-12 09:57:28.580005,2020-05-12 09:57:28.580005,1,2020-05-13 23:30:15.860005,...,0.0,NoAction,False,False,NaN,NaN,NaN,False,"INSERT INTO ""res_users_log"" (""id"", ""create_uid...",res_users_log
4,4,4,4,GenerateJobApplicationActivity_1.pcap,GenerateJobApplicationActivity,216,2020-05-12 09:57:28.729685,2020-05-12 09:57:28.729685,1,2020-05-13 23:55:12.809685,...,0.0,NoAction,False,False,IsNumber,authenticate,NaN,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33068,33068,11958,11958,ResumeReviewActivity_250.pcap,ResumeReviewActivity,729,2020-05-14 00:17:45.346366,2020-05-14 00:17:45.346366,250,2020-05-14 04:15:16.416366,...,0.0,NoAction,False,False,NaN,NaN,NaN,False,"UPDATE ""mail_message_res_partner_needaction_re...",mail_message_res_partner_needaction_rel
33069,33069,11959,11959,ResumeReviewActivity_250.pcap,ResumeReviewActivity,766,2020-05-14 00:17:45.361497,2020-05-14 00:17:45.361497,250,2020-05-14 04:17:47.741497,...,0.0,NoAction,False,False,IsNumber,execute_kw,hr.applicant_write_stage_id_2,True,None,None
33070,33070,11960,11960,ResumeReviewActivity_250.pcap,ResumeReviewActivity,774,2020-05-14 00:17:45.363203,2020-05-14 00:17:45.363203,250,2020-05-14 04:18:04.803203,...,0.0,NoAction,False,False,hr.applicant_archive_applicant,NaN,NaN,False,None,None
33071,33071,11961,11961,ResumeReviewActivity_250.pcap,ResumeReviewActivity,840,2020-05-14 00:17:45.427437,2020-05-14 00:17:45.427437,250,2020-05-14 04:28:47.207437,...,0.0,NoAction,False,False,NaN,NaN,NaN,False,"UPDATE ""hr_applicant"" SET ""active""=false,""writ...",hr_applicant


In [5]:
df_train["feat_cat"] = df_train["event_with_roles"] + df_train["selective_file_data"] + df_train["request_method_call"]

In [7]:
df_train["feat_cat"].value_counts()

feat_cat
Odoo Application->db Server/Mail Server:[PgsqlRequest:Simple query]: UPDATE: hr_applicant                                           2233
Odoo Application->End Point (HR Manager):[HttpResponse:HTTP/1.0 200 OK\r\n]IsNumberIsNumber                                         2135
Odoo Application->db Server/Mail Server:[PgsqlRequest:Simple query]: UPDATE: mail_message                                           2067
Odoo Application->db Server/Mail Server:[PgsqlRequest:Simple query]: UPDATE: mail_message_res_partner_needaction_rel                2067
Odoo Application->db Server/Mail Server:[PgsqlRequest:Simple query]: INSERT: mail_message                                           1908
                                                                                                                                    ... 
End Point (HR Manager)->Odoo Application:[HttpRequest:POST /xmlrpc/2/object HTTP/1.1\r\n]hr.applicant_write_stage_id_4execute_kw      39
End Point (HR Manager)->Odoo App

In [8]:
df_train[["event_with_roles","selective_file_data","request_method_call"]].value_counts()

event_with_roles                                                                                                      selective_file_data            request_method_call
Odoo Application->db Server/Mail Server:[PgsqlRequest:Simple query]: UPDATE: hr_applicant                                                                                   2233
Odoo Application->End Point (HR Manager):[HttpResponse:HTTP/1.0 200 OK\r\n]                                           IsNumber                       IsNumber               2135
Odoo Application->db Server/Mail Server:[PgsqlRequest:Simple query]: UPDATE: mail_message_res_partner_needaction_rel                                                        2067
Odoo Application->db Server/Mail Server:[PgsqlRequest:Simple query]: UPDATE: mail_message                                                                                   2067
Odoo Application->db Server/Mail Server:[PgsqlRequest:Simple query]: INSERT: mail_message                                  

In [14]:
df_train_r1["http_file_data"] = df_train_r1["MessageAttributes"].apply(lambda x: x.get("http.file_data", None))

In [15]:
df_train_r1[["MessageType_WithRole","http_file_data"]].value_counts()

MessageType_WithRole                                                                       http_file_data                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               